In [ ]:
%use roboquant(2.1.0)

// Still alpha, so need to import the backtest package manually 
import org.roboquant.backtest.*

In [ ]:
val feed = AvroFeed.sp500()

In [ ]:
// Define the search space
val space = RandomSearch(100)
space.add("ema1", 5 .. 100 step 2)
space.add("ema2", 5 .. 100 step 2)
space.add("rsi1", 5.0 .. 95.0 step 1.0)
space.add("rsi2", 5.0 .. 95.0 step 1.0)

In [ ]:
// Parameterize the strategy
fun getStrategy(params: Params) : Strategy {
    val ema1 = params.getInt("ema1")
    val ema2 = params.getInt("ema2")
    val rsi1 = params.getDouble("rsi1")
    val rsi2 = params.getDouble("rsi2")
    
    val s = TaLibStrategy()
    
    s.buy { 
        ema(it.close, ema1) > ema(it.close, ema2) && rsi(it.close, ema1) < rsi1 
    }

    s.sell {
        ema(it.close, ema1) < ema(it.close, ema2) && rsi(it.close, ema1) > rsi2    
    }
    
    return s
}

In [ ]:
// Define the Optimizer with Compound Annual Growth Rate
// as the score function
val opt = Optimizer(space, score = CAGR()) { params ->
    val s = getStrategy(params)
    Roboquant(s, logger = SilentLogger())
}

In [ ]:
// Run the optimization
val result = opt.train(feed, feed.timeframe)

// What is the best score
result.maxBy { it.score }

In [ ]:
// Show distribution of the scores
val data = result.map { it.score }.toDoubleArray()
HistogramChart(data)